In [1]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline



Run the redis server:
`redis-server`

make sure mysql runs

```
docker run --name=user_mysql_1 --env="MYSQL_ROOT_PASSWORD=root_password" -p 3306:3306 -d mysql:latest
```

make sure redis runs

```
redis-server
```


connect to mysql each redis worker


**addendum**: to stop 

```
/usr/local/bin/mysql.server stop

ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root_password';
```



In [2]:
# unit test
from redis import Redis
from rq import Queue
from rq import Retry

# needed
from parallel_hpo import connect_to_mysql, run_hpo_parallel, test_fnc

redis = Redis()
q = Queue(connection=redis)
job = q.enqueue(test_fnc, "hello!", retry=Retry(max=3))
job

No TPUs...


Job('29cb06aa-9ab2-4f48-8dab-49a0f37fdc11', enqueued_at=datetime.datetime(2022, 6, 21, 9, 22, 27, 920082))

In [3]:
# need to wait a bit before being able to see the result
job.result

So far you have enqueued the jobs: now you have to start the workers so that the jobs can be crunched!

```
rq worker high default low
```

You can set up multiple workers to have the HPO run in parallel.

In [4]:
from gdeep.search import HyperParameterOptimization

import numpy as np
import plotly.express as px
import torch
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim import SGD, Adam
from torch.utils.data import SubsetRandomSampler
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_betti_surfaces
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from sklearn import datasets

# RQ
from redis import Redis
from rq import Queue, Connection
from rq import Retry

from gdeep.models import FFNet
from gdeep.models import ModelExtractor
from gdeep.analysis.interpretability import Interpreter
from gdeep.utility.optimisation import SAMOptimizer
from gdeep.visualisation import persistence_diagrams_of_activations
from gdeep.trainer import Trainer
from gdeep.data.datasets import DatasetBuilder, DataLoaderBuilder
from gdeep.visualisation import Visualiser
from gdeep.utility import DEVICE
from gdeep.search import GiottoSummaryWriter



# Enqueuing the jobs

In the next section we enqueue te HPO and make sure that the workers are actively cruching the jobs! If more tan one worker is active, the job gets distributed!

In [9]:


# constants
USER = "root"
HOST = "127.0.0.1"
PSW = "root_password"

# connect to mysql
connect_to_mysql(USER, PSW, HOST)
print("DB connected")

# enqueue hpo jobs
hpo_job = q.enqueue(run_hpo_parallel, args=(USER, PSW, HOST), retry=Retry(max=3))
hpo_job2 = q.enqueue(run_hpo_parallel, args=(USER, PSW, HOST), retry=Retry(max=3))
print("jobs enqueued")

('example',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
DB connected
jobs enqueued
